In [1]:
import numpy as np
import pandas as pd
from google.cloud import storage
from sklearn.metrics import classification_report

In [2]:
class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

In [3]:
test_files_df = pd.read_csv('./test_files.csv')
fastai_df = pd.read_csv('fastai_test_results.csv')
test_files_df = test_files_df.merge(fastai_df, on='LOCAL_FILE_PATH')

In [4]:
client = storage.Client()
automl_df = pd.DataFrame()
for blob in client.list_blobs('cv-with-jake-fashionmnist', prefix='automl_results/prediction-fashion_mnist_eval-2022-03-01T05:59:11.082931Z'):
    file_name = f'gs://cv-with-jake-fashionmnist/{blob.name}'
    automl_df = pd.concat([automl_df, pd.read_json(file_name, lines=True)])

In [5]:
automl_df['AUTOML_PRED'] = automl_df['prediction'].apply(lambda x: x['displayNames'][0])
automl_df['AUTOML_PRED'] = pd.to_numeric(automl_df['AUTOML_PRED'])

In [6]:
automl_df['GCS_FILE_PATH'] = automl_df['instance'].apply(lambda x: x['content'])

In [7]:
automl_df = automl_df[['AUTOML_PRED', 'GCS_FILE_PATH']]

In [8]:
automl_df.head()

,AUTOML_PRED,GCS_FILE_PATH
0,4,gs://cv-with-jake-fashionmnist/test/img_6700.jpg
1,7,gs://cv-with-jake-fashionmnist/test/img_4284.jpg
2,1,gs://cv-with-jake-fashionmnist/test/img_517.jpg
3,7,gs://cv-with-jake-fashionmnist/test/img_7772.jpg
4,8,gs://cv-with-jake-fashionmnist/test/img_2696.jpg


In [9]:
test_files_df = test_files_df.merge(automl_df, on='GCS_FILE_PATH')

In [10]:
automl_preds = test_files_df['AUTOML_PRED']
fastai_preds = test_files_df['FASTAI_PRED']
labels = test_files_df['LABEL']

In [11]:
# generate automl report
automl_report = classification_report(automl_preds, labels, digits=4, target_names=class_names)

In [12]:
fastai_report = classification_report(fastai_preds, labels, digits=4, target_names=class_names)

In [13]:
print(automl_report)

              precision    recall  f1-score   support

 T-shirt/top     0.9110    0.8514    0.8802      1070
     Trouser     0.9820    0.9980    0.9899       984
    Pullover     0.8470    0.8851    0.8656       957
       Dress     0.9130    0.9157    0.9144       997
        Coat     0.8750    0.8124    0.8426      1077
      Sandal     0.9480    0.9875    0.9673       960
       Shirt     0.6980    0.7670    0.7309       910
     Sneaker     0.9840    0.9094    0.9452      1082
         Bag     0.9920    0.9880    0.9900      1004
  Ankle boot     0.9440    0.9844    0.9638       959

    accuracy                         0.9094     10000
   macro avg     0.9094    0.9099    0.9090     10000
weighted avg     0.9115    0.9094    0.9098     10000



In [14]:
print(fastai_report)

              precision    recall  f1-score   support

 T-shirt/top     0.8800    0.8653    0.8726      1017
     Trouser     0.9930    0.9910    0.9920      1002
    Pullover     0.8910    0.8787    0.8848      1014
       Dress     0.9200    0.9455    0.9326       973
        Coat     0.8910    0.8559    0.8731      1041
      Sandal     0.9850    0.9830    0.9840      1002
       Shirt     0.7570    0.8002    0.7780       946
     Sneaker     0.9690    0.9556    0.9623      1014
         Bag     0.9870    0.9840    0.9855      1003
  Ankle boot     0.9660    0.9777    0.9718       988

    accuracy                         0.9239     10000
   macro avg     0.9239    0.9237    0.9237     10000
weighted avg     0.9246    0.9239    0.9241     10000

